In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import time as time
import re
from random import randint
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import logging

In [3]:
import os

In [4]:
import sys
sys.path.append("../../code")
#sys.path.append("../../loaders")
import loaders.cityprotect as cp

Concatenate and pandaize data downloaded from CityProtect.
Given a folder, find all the files ending with "Los_Altos_Police_Department_report.csv", concatenate, and sort by date.

In [5]:
def geocode(geolocator, addr, sleep_sec):
    try:
        return geolocator.geocode(addr)
    except GeocoderTimedOut:
        logging.info('TIMED OUT: GeocoderTimedOut: Retrying...')
        time.sleep(randint(1*100,sleep_sec*100)/100)
        return geolocator.reverse_geocode(geolocator, latlon, sleep_sec)
    except GeocoderServiceError as e:
        logging.info('CONNECTION REFUSED: GeocoderServiceError encountered.')
        logging.error(e)
        return None
    except Exception as e:
        logging.info('ERROR: Terminating due to exception {}'.format(e))
        return None

In [16]:
import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.geocoders import Photon
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
user_agent = 'user_me_{}'.format(randint(10000,99999))
geolocator = Photon(user_agent=user_agent, scheme='http')



In [17]:
location = geolocator.reverse("48.8588443, 2.2943506")

print(location.address)
print (location.raw)

Pilier Nord Caisse, Quai Jacques Chirac, 75007, Quai Jacques Chirac, Paris, Île-de-France, France
{'geometry': {'coordinates': [2.2943973500000054, 48.8587893], 'type': 'Point'}, 'type': 'Feature', 'properties': {'osm_id': 335101035, 'extent': [2.2943659, 48.8588097, 2.2944288, 48.8587689], 'country': 'France', 'city': 'Paris', 'countrycode': 'FR', 'postcode': '75007', 'locality': 'Quartier du Gros-Caillou', 'type': 'house', 'osm_type': 'W', 'osm_key': 'shop', 'street': 'Quai Jacques Chirac', 'district': 'Paris', 'osm_value': 'ticket', 'name': 'Pilier Nord Caisse', 'state': 'Île-de-France'}}


In [42]:
bob = geolocator.geocode("1728 Selig Ln, Los Altos, CA")

In [54]:
print(bob.raw["properties"]["city"])

Los Altos


In [55]:
bob.address

'Selig Lane, 94807, Los Altos, California, United States'

In [7]:
previous_df = pd.read_pickle("../../data/clean_data/SCCSheriff_2017_2022_inferredCity.pkl")

In [8]:
previous_df["date"].max()

Timestamp('2022-12-31 23:58:48')

In [9]:
dept = "Santa Clara County Office of the Sheriff"
df = cp.cityprotect("../../data/raw_data/{}".format(dept.replace(" ", "_")),
                                 start_date = previous_df["date"].max(),
                                  end_date = datetime.now() )

Starting: 2022-12-31 23:58:48
Ending:   2023-10-13 14:25:07.097616
Loading from 26 files.
Data file ../../data/raw_data/Santa_Clara_County_Office_of_the_Sheriff\Apr_Jun_2017_Santa_Clara_County_Office_of_the_Sheriff_report.csv is empty.
Data file ../../data/raw_data/Santa_Clara_County_Office_of_the_Sheriff\Jan_Mar_2017_Santa_Clara_County_Office_of_the_Sheriff_report.csv is empty.


In [19]:
df["date"].min()

Timestamp('2022-12-31 23:58:48')

In [20]:
df["inferredCity"]="unknown"

In [37]:
row = df.iloc[125]

In [38]:
row

ccn                                                          S230010338
date                                                2023-01-02 04:52:30
updateDate                                          2023-01-02 17:39:29
city                                                 SANTA CLARA COUNTY
state                                                                CA
postalCode                                                            .
blocksizedAddress                                             BERRY WAY
incidentType                                                DISTURBANCE
parentIncidentType                                      Quality of Life
narrative             Call Type: 415    <br>Description: DISTURBANCE...
inferredCity                                                    unknown
Name: 174833, dtype: object

In [39]:
place = re.sub("BLOCK", "", row["blocksizedAddress"].upper()) + ", SANTA CLARA COUNTY, CA"

In [40]:
location = geocode(geolocator, place, 1)

In [41]:
location

Location(Berry Way, 95051, Santa Clara, California, United States, (37.3606331, -121.9951343, 0.0))

In [56]:
counter = 0
for index, row in df[df["inferredCity"] == "unknown"].iterrows():
    if row["inferredCity"] == "unknown":
        print(row)
        try:
            place = re.sub("BLOCK", "", row["blocksizedAddress"].upper()) + ", SANTA CLARA COUNTY, CA"
        except:
            print("Failed to scan: {}".format(row["blocksizedAddress"]))
            continue
        location = geocode(geolocator, place, 1)   
            
        counter = counter + 1
        if location != None:
            if type(location) == str:
                display_name = [s.strip() for s in location.raw["display_name"].split(",")]                         
                try:
                    county_idx = display_name.index("Santa Clara County")
                except ValueError:
                    print("SCC not found: {}".format(display_name))
                    df.at[index, "inferredCity"] = "not_found"
                    continue
                df.at[index, "inferredCity"] = display_name[county_idx-1]   
                df.at[index, "postalCode"] = display_name[-2]                            
                print("\n{} {}: {}, {}".format(counter, location.raw["display_name"], df.at[index,"inferredCity"], df.at[index, "postalCode"]))
            else:
                df.at[index, "inferredCity"] = "no resolution"
                print("NR: {} ".format(counter), end="")
        elif type(location) == geopy.location.Location:
            df.at[index, "inferredCity"] = location.raw["properties"]["city"]
            df.at[index, "postalCode"] = location.raw["properties"]["postcode"]
            print("\n{}: {}, {}".format(counter, df.at[index,"inferredCity"], df.at[index, "postalCode"]))
        if counter/1000 == round(counter/1000):
            print("Saving...")
            df.to_pickle("SCCSheriff.pkl")
    else:
        print(".", end="")


ccn                                                          S223650282
date                                                2022-12-31 23:58:48
updateDate                                          2023-01-01 09:05:07
city                                                 SANTA CLARA COUNTY
state                                                                CA
postalCode                                                            .
blocksizedAddress                               14000 Block LOMA RIO DR
incidentType                                       PHONE UR OFFICE, OR:
parentIncidentType                                                Other
narrative             Call Type: 1021   <br>Description: PHONE UR OF...
inferredCity                                                    unknown
Name: 324514, dtype: object
NR: 1 ccn                                                          L223650085
date                                                2023-01-01 00:15:09
updateDate                    

KeyboardInterrupt: 

In [ ]:
df.to_pickle("SCCSheriff.pkl")

In [ ]:
df["inferredCity"].value_counts()

In [ ]:
df[df["inferredCity" == "unknown"]]

        try: 
            location = geolocator.geocode(place)
        except GeocoderTimedOut as e:
            print("Location error: {}".format(e))
            naptime = 3+randint(100)
            print("Sleeping {} seconds.".format(naptime))
            time.sleep(naptime)
            user_agent = 'user_me_{}'.format(randint(10000,99999))
            geolocator = Nominatim(user_agent=user_agent)
            location = geolocator.geocode(place)
